## Installations

In [ ]:
!pip install -U weaviate-client llama-index

In [1]:
# let's catch some logs
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Connect to Weaviate

In [2]:
import weaviate
from weaviate import classes as wvc
client = weaviate.connect_to_local()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


In [3]:
# lets check the connection getting the server version
print(f"Client: {weaviate.__version__}, Server: {client.get_meta().get('version')}")

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
Client: 4.9.3, Server: 1.26.7


### Create Collection

In [4]:

# lets make sure we do not have this collection first
client.collections.delete(["WeaviateBlogPost", "HuggingFaceBlogPost"])

collection_weaviate_blog = client.collections.create(
    "WeaviateBlogPost",
    description="Blog post from the Weaviate website",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    properties=[
        wvc.config.Property(
            name="content", data_type=wvc.config.DataType.TEXT
        )
    ]
)

collection_huggingface_blog = client.collections.create(
    "HuggingFaceBlogPost",
    description="Blog post from the HuggingFace website",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small"
    ),
    properties=[
        wvc.config.Property(
            name="content", data_type=wvc.config.DataType.TEXT
        )
    ]
)

INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"


### Load in Data

In [5]:
# lets import llama lib
from llama_index.core import download_loader
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
import os

/Users/dudanogueira/dev/weaviate/recipes/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [6]:
# you must have the API key as envi variable
#os.environ["OPENAI_API_KEY"] = "sk-..."

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.embed_model = embed_model

SimpleWebPageReader = download_loader("SimpleWebPageReader")

loader = SimpleWebPageReader(html_to_text=True)
WeaviateBlog = loader.load_data(urls=['https://weaviate.io/blog/pq-rescoring'])
WeaviateBlog_vector_store = WeaviateVectorStore(weaviate_client=client, index_name="WeaviateBlogPost", text_key="content")
WeaviateBlog_storage_context = StorageContext.from_defaults(vector_store=WeaviateBlog_vector_store)
WeaviateBlogIndex = VectorStoreIndex.from_documents(WeaviateBlog, storage_context=WeaviateBlog_storage_context)

HuggingFaceBlog = loader.load_data(urls=['https://huggingface.co/blog/ram-efficient-pytorch-fsdp'])
HuggingFaceBlog_vector_store = WeaviateVectorStore(weaviate_client=client, index_name="HuggingFaceBlogPost", text_key="content")
HuggingFaceBlog_storage_context = StorageContext.from_defaults(vector_store=HuggingFaceBlog_vector_store)
HuggingFaceBlogIndex = VectorStoreIndex.from_documents(HuggingFaceBlog, storage_context=HuggingFaceBlog_storage_context)

/var/folders/4j/js2lp8b17zs2cvk9v9pl26cc0000gn/T/ipykernel_26902/1452062062.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  SimpleWebPageReader = download_loader("SimpleWebPageReader")


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"


## Create Index

In [7]:
from llama_index.core.schema import IndexNode

summaries = {
    "Weaviate": "This node provides blog posts from Weaviate, a Vector Database.",
    "HuggingFace": "This node provides blog posts from HuggingFace, tools for training Machine Learning models."
}

df_nodes = [
    IndexNode(text=summaries["Weaviate"], index_id="WeaviateBlogs"),
    IndexNode(text=summaries["HuggingFace"], index_id="HuggingFaceBlogs")
]

WeaviateBlogQueryEngine = WeaviateBlogIndex.as_query_engine()
HuggingFaceBlogQueryEngine = HuggingFaceBlogIndex.as_query_engine()

df_id_query_engine_mapping = {
    "WeaviateBlogs": WeaviateBlogQueryEngine,
    "HuggingFaceBlogs": HuggingFaceBlogQueryEngine
}

Tool_Description_Index = VectorStoreIndex(df_nodes)
Tool_Retriever = Tool_Description_Index.as_retriever(similarity_top_k=1)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Build Recursive Retriever

In [8]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": Tool_Retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True
)

response_synthesizer = get_response_synthesizer(
    response_mode="compact"
)

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [9]:
response = query_engine.query("What is Product Quantization?").response

Retrieving with query id None: What is Product Quantization?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieved node with id, entering: WeaviateBlogs
Retrieving with query id WeaviateBlogs: What is Product Quantization?
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/WeaviateBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got response: Product Quantization is a method used to compress vectors, enabling users to reduce memory requirements. It works b

In [10]:
response = query_engine.query("What does FSDP do?").response

Retrieving with query id None: What does FSDP do?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieved node with id, entering: HuggingFaceBlogs
Retrieving with query id HuggingFaceBlogs: What does FSDP do?
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HuggingFaceBlogPost "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got response: FSDP is a paradigm where the optimizer states, gradients, and parameters are sharded across devices. During the forward

In [11]:
client.close()